# 任务四 —— Transformer —— "Attention Is All You Need" 论文精读与实践

## 任务三：代码实现

1. 实现 ScaledDotProductAttention

In [1]:
import torch
import torch.nn as nn
import math

In [ ]:
def scaled_dot_product_attention(q, k, v, mask=None):
     # --- 请你完成这部分代码 ---
    y = torch.matmul(q, k.transpose(-2,-1))/ math.sqrt(k.size(-1))
    if mask is not None:
        y = y.masked_fill(mask == 0, float('-inf'))
    attention_weights = torch.softmax(y, dim=-1) 
    output = torch.matmul(attention_weights, v)
    return output, attention_weights

这里说明一下mask机制，因为在原文中并没有过多介绍。mask机制主要分为paddingmask和casualmask，前者用于屏蔽\<PAD\>,后者用于在Decoder中屏蔽未来位置。在代码实现中，我们会把mask矩阵中为0的地方替换为-inf，因为这样对应softmax趋于0。在使用时通过传入的mask矩阵类型确定采用何种方式，如paddingmask就传入标记\<PAD\>的矩阵，casualmask就传入下三角矩阵。

2. 构建一个 SelfAttention 模块

In [3]:
class SimpleSelfAttention(nn.Module):
    def __init__(self, embed_dim, head_dim):
        super().__init__()
        self.q_proj = nn.Linear(embed_dim, head_dim)
        self.k_proj = nn.Linear(embed_dim, head_dim)
        self.v_proj = nn.Linear(embed_dim, head_dim)
    
    def forward(self, x):
        # --- 请你完成这部分代码 ---
        # 1. 将 x 传入线性层得到 q, k, v
        # 2. 调用 scaled_dot_product_attention
        # 3. 返回输出结果
        q = self.q_proj(x)
        k = self.k_proj(x)
        v = self.v_proj(x)
        output, attention_weights = scaled_dot_product_attention(q, k, v)
        return output, attention_weights

3. 验证实现：
    这里采用原论文中的512作为embedding_dim，64作为head_dim。输入的序列长度为20，批量大小是10

In [ ]:
# --- 验证部分 ---
# 创建实例和输入数据，并检查输出形状
embed_dim = 512
head_dim = 64
attention = SimpleSelfAttention(embed_dim, head_dim)
x = torch.rand(10, 20, embed_dim)  # (batch_size, seq_len, embed_dim)，当维度较多时写注释相当重要
output, attn_weights = attention(x)
print(output.shape)  # 应该是 (10, 20, head_dim)
print(attn_weights.shape)  # 应该是 (10, 20, 20) 

torch.Size([10, 20, 64])
torch.Size([10, 20, 20])
